In [1]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("restaurant-review-average")
sc = SparkContext(conf=conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/25 20:49:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
pairs = sc.parallelize([
    1,2,3,4,2,4,1
]).map(lambda x : (x, x**x))
pairs.collect()

[(1, 1), (2, 4), (3, 27), (4, 256), (2, 4), (4, 256), (1, 1)]

In [4]:
# 기본 파티션 확인
pairs.glom().collect()

[[(1, 1), (2, 4), (3, 27), (4, 256), (2, 4), (4, 256), (1, 1)]]

In [5]:
# 파티션을 2개로 나누기
pairs.partitionBy(2).glom().collect() #짝수끼리, 홀수끼리 잘 모였다

[[(2, 4), (4, 256), (2, 4), (4, 256)], [(1, 1), (3, 27), (1, 1)]]

In [8]:
# 개발자가 직접 파티션 배치 함수를 만들 수 있다.(Hash Function Custom)
pairs.partitionBy(2, lambda x : x%2).glom().collect()

[[(2, 4), (4, 256), (2, 4), (4, 256)], [(1, 1), (3, 27), (1, 1)]]

In [9]:
# 개발자가 직접 파티션 배치 함수를 만들 수 있다.(Hash Function Custom)
pairs.partitionBy(2, lambda x : x%3).glom().collect()

[[(2, 4), (3, 27), (2, 4)], [(1, 1), (4, 256), (4, 256), (1, 1)]]

파티션을 만든 후에 `persist()`를 하지 않으면 다음 연산에 rdd가 사용 될 때마다 파티션을 만드는 코드가 계속 반복된다.
- 셔플링이 반복적으로 일어날 수 있다.

In [12]:
pairs.partitionBy(2, lambda x : x%3).persist().glom().collect() # 이게 정석이다

[[(2, 4), (3, 27), (2, 4)], [(1, 1), (4, 256), (4, 256), (1, 1)]]

In [13]:
sc.stop()